In [2]:
import os, sys
import ROOT
sys.path.append("../Utils/")
sys.path.append("../Config/")
from xconfig import *
from fillReport import *
import xutils
import math


#Get detector info (index, detector No, calib factor)
cfg = Config('../Config/detectors.ini')
dummy = cfg.get_option('Detectors')
detInfo = {}
for key in dummy.keys():
    detInfo[key] = dummy[key].split()

fillReport = FillReport("../Config/FillReport.xls")

###TChain for Offline lumi combined

In [29]:
t = ROOT.TChain("t")
t.Add("/scr1/RadMonLumi/2016/OfflineLumi/Radmon_normtag_BRIL/*root");
warm = ROOT.TCut("tstamp - fillStable>60*60")
stable = ROOT.TCut("beamStatus == \"STABLE BEAMS\"")
hfoc = ROOT.TCut("lumiSource==\"HFOC\"")
plt = ROOT.TCut("lumiSource==\"PLTZERO\"")
ratio = ROOT.TCut("lumi/rates[12] > 0 && lumi/rates[12]<3")

#Cuts
deltaWarming = 60*60


###TChain for Online lumi combined

In [3]:
t = ROOT.TChain("t")
t.Add("/scr1/RadMonLumi/2016/OnlineLumi/RadmonLumiFills/*.root");
stable = ROOT.TCut("tstamp>fillStable&&tstamp<fillEnd")
deltaWarming = 60*60

###Ratio of the rates to one reference detector 

In [9]:
fillsRange = [4850, 5250]
detRef = "PNIB"
indRef = int(detInfo[detRef.lower()][0])
dets = ["PFXT", "PFIB", "PFIT", "PNXT", "PNIB", "PNIT", "MFXT", "MFIB", "MFIT", 
        "MNXT", "MNIB", "MNIT"]
histos = {}
for det in dets:
    try:
        del histos[det]
    except:
        pass
    histos[det] = ROOT.TH2D(det, det.upper() + "/" + detRef.upper(),
                  (fillsRange[1] - fillsRange[0]), fillsRange[0], fillsRange[1], 1000, 0., 3.)

# Filling
for i in range(0, t.GetEntries()) :
    nb = t.GetEntry(i)
    if nb < 0:
        continue
    
    
#     if t.beamStatus[0:12] != "STABLE BEAMS":
#         continue
    if t.tstamp > t.fillEnd:
        continue
    if t.tstamp - t.fillStable < deltaWarming:
        continue
    
#     if i > 10000:    break
        
    for det in dets:
        index = int(detInfo[det.lower()][0])
        if t.rates[index>0] and t.rates[indRef] > 0.:
            histos[det].Fill(t.fill, t.rates[index]/t.rates[indRef] )

profx = {}
for det in dets:
    try:
        profx[det].Delete()
    except:
        pass
    profx[det] = histos[det].ProfileX(det.lower()+"x", 1, -1, "s")
    profx[det].SetTitle(histos[det].GetTitle())


### Who is to blame for the drift (Ratio to the average of selected detectors)

In [3]:
fillsRange = [4850, 5250]
detRef = ["PFIT", "PFIB", "PNIT", "PNIB", "MNIB", "MNIT"]
dets = ["PFXT", "PFIB", "PFIT", "PNXT", "PNIB", "PNIT", "MFXT", "MFIB", "MFIT", 
        "MNXT", "MNIB", "MNIT"]
histos = {}
for det in dets:
    try:
        del histos[det]
    except:
        pass
    histos[det] = ROOT.TH2D(det, det.upper() + "/(" + "+".join(detRef) + ")",
                  (fillsRange[1] - fillsRange[0]), fillsRange[0], fillsRange[1], 1000, 0., 3.)

# Filling
for i in range(0, t.GetEntries()) :
    nb = t.GetEntry(i)
    if nb < 0:
        continue
    
    
#     if t.beamStatus[0:12] != "STABLE BEAMS":
#         continue
    if t.tstamp > t.fillEnd:
        continue
    if t.tstamp - t.fillStable < deltaWarming:
        continue
    
#     if i > 10000:    break

    averef = 0
    nref = 0
    for ref in detRef:
        indRef = int(detInfo[ref.lower()][0])
        cfRef = float(detInfo[ref.lower()][2])
        if t.rates[indRef] > 0:
            averef += t.rates[indRef] * cfRef
            nref += 1
    if nref > 0:
        averef /= nref
        
    for det in dets:
        index = int(detInfo[det.lower()][0])
        cf = float(detInfo[det.lower()][2])

        value = 0.
        
        if averef > 0:    value = t.rates[index] * cf / averef
        histos[det].Fill(t.fill, value)

profx = {}
for det in dets:
    try:
        profx[det].Delete()
    except:
        pass
    profx[det] = histos[det].ProfileX(det.lower()+"x", 1, -1, "s")
    profx[det].SetTitle(histos[det].GetTitle())


###Who is to blame for the drift (Ratio to the average of selected detectors) vs TIMESTAMP (huge memory consumption - no need to use)

In [126]:
fillsRange = [4890, 5247]

tstart = fillReport.getFillStableTime(fillsRange[0])
tsend = fillReport.getFillEndTime(fillsRange[1])
secsPerBin = 230
nbins = int((tsend - tstart)/secsPerBin)

detRef = ["PFIT", "PFIB", "PNIT", "PNIB", "MNIB", "MNIT"]
dets = ["PFXT", "PFIB", "PFIT", "PNXT", "PNIB", "PNIT", "MFXT", "MFIB", "MFIT", 
        "MNXT", "MNIB", "MNIT"]
histos = {}
for det in dets:
    try:
        del histos[det]
    except:
        pass
    histos[det] = ROOT.TH2D(det, det.upper() + "/(" + "+".join(detRef) + ")",
                  nbins, tstart, tsend, 1000, 0., 3.)

# Filling
for i in range(0, t.GetEntries()) :
    nb = t.GetEntry(i)
    if nb < 0:
        continue
    
    
#     if t.beamStatus[0:12] != "STABLE BEAMS":
#         continue
    if t.tstamp > t.fillEnd:
        continue
    if t.tstamp - t.fillStable < deltaWarming:
        continue
    
#     if i > 10000:    break

    averef = 0
    nref = 0
    for ref in detRef:
        indRef = int(detInfo[ref.lower()][0])
        cfRef = float(detInfo[ref.lower()][2])
        if t.rates[indRef] > 0:
            averef += t.rates[indRef] * cfRef
            nref += 1
    if nref > 0:
        averef /= nref
        
    for det in dets:
        index = int(detInfo[det.lower()][0])
        cf = float(detInfo[det.lower()][2])

        value = 0.
        
        if averef > 0:    value = t.rates[index] * cf / averef
        histos[det].Fill(t.tstamp, value)

profx = {}
for det in dets:
    try:
        profx[det].Delete()
    except:
        pass
    profx[det] = histos[det].ProfileX(det.lower()+"x", 1, -1, "s")
    profx[det].SetTitle(histos[det].GetTitle())


In [14]:
c = ROOT.TCanvas("c", "c", 800, 800)
c.Divide(1,2)
detector = "MFXT"
c.cd(1)
histos[detector].Draw()
c.cd(2)
profx[detector].Draw()

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: c


###Lumi plots (from online data)

In [4]:
t.Draw("pltZeroLumi:hfLumi", stable.GetTitle())

180413L

###HF pltZero/Lumi vs fillno at stable beam (Online)

In [24]:
fillsRange = [4850, 5250]

hlumi = ROOT.TH2D("hlumi", "Lumi. PltZero/HF vs Fill",
                  (fillsRange[1] -  fillsRange[0]), fillsRange[0], fillsRange[1], 1000, 0., 5.)

# Filling
for i in range(0, t.GetEntries()) :
    nb = t.GetEntry(i)
    if nb < 0:
        continue
    if t.tstamp > t.fillEnd or t.tstamp < t.fillStable:
        continue
    
    if t.fill == 5194:   continue
    value = 0.
    if t.hfLumi > 0 and t.pltZeroLumi >0:
        value = t.pltZeroLumi/t.hfLumi
        hlumi.Fill(t.fill, value)
        
try:
    plumi.Delete()
except:
    pass
plumi = hlumi.ProfileX("plumi", 1, -1, "s")

c = ROOT.TCanvas("c", "c", 800, 800)
c.Divide(1,2)
c.cd(1)
hlumi.Draw()
c.cd(2)
plumi.Draw()


TROOT::Append:0: RuntimeWarning: Replacing existing TH1: hlumi (Potential memory leak).
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: c


In [25]:

c = ROOT.TCanvas("c", "c", 800, 800)
c.Divide(1,2)
c.cd(1)
hlumi.Draw()
c.cd(2)
plumi.Draw()


TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: c


### Mean neutron fluence vs lumi 

In [5]:
fillsRange = [4850, 5250]
dets = ["PFIT", "PFIB", "PNIT", "PNIB", "MNIB", "MNIT"]

nbins = 1000

hrfill = ROOT.TH2D("hrfill", "mean flux over lumi", 
                  (fillsRange[1] - fillsRange[0]), fillsRange[0], fillsRange[1], 1000, 0., 7.)
hflxlumi = ROOT.TH2D("hflxlumi", "mean flux vs lumi", nbins, 0., 15000., nbins, 0., 60000.)
hflxplt = ROOT.TH2D("hflxplt", "mean flux vs PLT ZERO lumi", nbins, 0., 15000., nbins, 0., 60000.)
hflxhf = ROOT.TH2D("hflxhf", "mean flux vs HFOC lumi", nbins, 0., 15000., nbins, 0., 60000.)


# Filling
for i in range(0, t.GetEntries()) :
    nb = t.GetEntry(i)
    if nb < 0:
        continue
    
    
    if t.beamStatus[:-1] != "STABLE BEAMS":
        continue
    if t.tstamp > t.fillEnd:
        continue
    if t.tstamp - t.fillStable < deltaWarming:
        continue
    
#     if i > 10000:    break

    meanflux = 0
    ndet = 0
    for det in dets:
        index = int(detInfo[det.lower()][0])
        cf = float(detInfo[det.lower()][2])
        if t.rates[index] > 0:
            meanflux += t.rates[index] * cf
            ndet += 1
    if ndet > 0:
        meanflux /= ndet
    
    if t.lumi > 0:
        hrfill.Fill(t.fill, meanflux/t.lumi)
        hflxlumi.Fill(t.lumi, meanflux)
#         hfoc = ROOT.TCut("lumiSource==\"HFOC\"")
# plt = ROOT.TCut("lumiSource==\"PLTZERO\"")
        if t.lumiSource[:-1] == "HFOC":
            hflxhf.Fill(t.lumi, meanflux)
        if t.lumiSource[:-1] == "PLTZERO":
            hflxplt.Fill(t.lumi, meanflux)

prfill = hrfill.ProfileX("prfill", 1, -1, "s")
# prfill.SetTitle(hrfill.GetTitle())


AttributeError: 'TChain' object has no attribute 'beamStatus'

In [35]:
c = ROOT.TCanvas("c", "c", 1000, 800)
c.Divide(1,3)
c.cd(1)
hflxlumi.Draw()
c.cd(2)
hrfill.Draw()
c.cd(3)
prfill.Draw()


TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: c


In [56]:
ROOT.gStyle.SetOptStat(0000);
c = ROOT.TCanvas("c", "c", 800, 600)
hflxplt.SetLineColor(ROOT.kRed)
hflxplt.SetMarkerColor(ROOT.kRed)
hflxplt.Draw()
hflxhf.SetLineColor(ROOT.kBlue)
hflxhf.SetMarkerColor(ROOT.kBlue)
hflxhf.Draw("same")

0000000000000100
